In [1]:
import csv
import pandas as pd
import mysql.connector as mysql
import mysql.connector.errors as errors

In [2]:
def connect_to_db(username, password, host, database):
    try:
        conn = mysql.connect(user=username, password=password, host=host, database=database)
    except errors.OperationalError as e:
        raise e
    else:
        print('Connected')
    return conn

In [3]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos1(
                        video_id VARCHAR(255) PRIMARY KEY, 
                        video_title TEXT, 
                        upload_date DATE DEFAULT (CURRENT_DATE), 
                        view_count INTEGER, 
                        like_count INTEGER, 
                        comment_count INTEGER)""")
    
    curr.execute(create_table_command)

In [4]:
def check_if_video_exists(curr, video_id):
    query = ("SELECT video_id FROM VIDEOS1 WHERE video_id = %s")
    curr.execute(query, (video_id,))
    return curr.fetchone() is not None

In [5]:
def update_row(curr, video_id, video_title, view_count, like_count, comment_count):
    query = ("""UPDATE videos1
            SET video_title = %s,
            view_count = %s, 
            like_count = %s, 
            comment_count = %s
            WHERE video_id = %s;""")
    
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

In [6]:
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=["video_id", "video_title", "upload_date",
                                   "view_count", "like_count", "comment_count"])

    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):
            update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['comment_count'])
        else:
            tmp_df = tmp_df.append(row)
    return tmp_df

In [7]:
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos1 (video_id, video_title, upload_date, view_count, like_count, comment_count)
            VALUES(%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)

In [8]:
def append_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['comment_count'])

In [9]:
df = pd.read_csv('channel_info.csv', index_col=0)
df.head()

df = df.fillna(-1)
df.isna().mean()

video_id         0.0
video_title      0.0
upload_date      0.0
view_count       0.0
like_count       0.0
comment_count    0.0
dtype: float64

In [10]:
username = "root"
password = "philippnesterov3"
host = "localhost"
database = "channel_info"
conn = connect_to_db(username, password, host, database)

Connected


In [11]:
curr = conn.cursor()

In [12]:
create_table(curr)

In [13]:
new_video_df = update_db(curr, df)

In [14]:
append_from_df_to_db(curr, new_video_df)

In [15]:
conn.commit()